<a href="https://colab.research.google.com/github/AprendizajeProfundo2021/trabajo-equipo7/blob/main/Dataset_segmentaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparando dataset

### Tablas

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN/

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 31.92 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [ ]:
%%capture
!wget https://raw.githubusercontent.com/holms-ur/fine-tuning/master/code/mask-rcnn/predict.py -O predict.py
!wget https://www.dropbox.com/s/dcl53rl3xqndfdx/mask_rcnn_tablebank_cfg_0002.h5?dl=0 -O mask_rcnn_tablebank_cfg_0002.h5

In [ ]:
cd Mask_RCNN/

/content/Mask_RCNN


In [ ]:
!pip install . --user

Processing /content/Mask_RCNN
  Created wheel for mask-rcnn: filename=mask_rcnn-2.1-cp37-none-any.whl size=56924 sha256=7117aadffbbc0f4132aec0d7808bbe052222751a9e7e43fcb4b24f66419df5b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-wfybnxol/wheels/78/da/b4/63931b0b9847ef98ac88d2b032817246adcf667c5da577133f
Successfully built mask-rcnn


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
%%capture
from predict import predict,showImage
from google.colab import files
import cv2  

## Descargando banco de datos

In [ ]:
import numpy as np
import os,shutil

In [ ]:
%%capture
!wget http://www.tamirhassan.com/html/files/icdar2013-competition-dataset-with-gt.zip -O Marmotdataset.zip
!unzip Marmotdataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
os.mkdir('dataset1')
%cd dataset1
!unzip /content/drive/MyDrive/Marmot_data.zip
%cd /content

In [ ]:
%cd /content

/content


## Procesando datos de segmentación de columnas
- Preparado para la segmentación de columnas

In [ ]:
T=['train','test']
subT=['images','labels']

os.mkdir('dataset') 
for x in T:
  os.mkdir('dataset/'+x)
  for y in subT: 
    os.mkdir('dataset/'+x+'/'+y)

In [ ]:
import xml.etree.ElementTree as ET
import os
import numpy as np
from PIL import Image

In [ ]:
def sameTable(ymin_1, ymin_2, ymax_1, ymax_2):
    min_diff = abs(ymin_1 - ymin_2)
    max_diff = abs(ymax_1 - ymax_2)

    if min_diff <= 5 and max_diff <=5:
        return True
    elif min_diff <= 4 and max_diff <=7:
        return True
    elif min_diff <= 7 and max_diff <=4:
        return True
    return False

In [ ]:
def ajustar(mask):
  W = mask.shape[1]
  for i in range(0,W):
    if True in list(np.unique(mask[:,i]==255)): mask[:,i] = mask[:,i] + 255

In [ ]:
directory = './dataset1/'
k = 0
for file in os.listdir(directory):
    
    filename = os.fsdecode(file)
    # Find all the xml files
    if filename.endswith(".xml"):
        filename = filename[:-4]
        # Parse xml file
        tree = ET.parse('./dataset1/' + filename + '.xml')
        root = tree.getroot()
        size = root.find('size')
        # Parse width
        width = int(size.find('width').text)
        height = int(size.find('height').text)
        # Create grayscale image array
        col_mask = np.zeros((height, width), dtype=np.int32)
        table_mask = np.zeros((height, width), dtype = np.int32)
        got_first_column = False
        i=0
        table_xmin = 10000
        table_xmax = 0
        table_ymin = 10000
        table_ymax = 0
        for column in root.findall('object'):
            bndbox = column.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            col_mask[ymin:ymax, xmin:xmax] = 255
                            
            if got_first_column:
                if sameTable(prev_ymin, ymin, prev_ymax, ymax) == False:
                    i+=1
                    got_first_column = False
                    table_mask[table_ymin:table_ymax, table_xmin:table_xmax] = 255
                    
                    table_xmin = 10000
                    table_xmax = 0
                    table_ymin = 10000
                    table_ymax = 0
                    
            if got_first_column == False:
                got_first_column = True
                first_xmin = xmin
                
            prev_ymin = ymin
            prev_ymax = ymax
            
            table_xmin = min(xmin, table_xmin)
            table_xmax = max(xmax, table_xmax)
            
            table_ymin = min(ymin, table_ymin)
            table_ymax = max(ymax, table_ymax)
        table_mask[table_ymin:table_ymax, table_xmin:table_xmax] = 255
        if k%5 == 0 and filename != '10.1.1.1':
          path_file='/content/dataset/test/'
        if k%5 != 0 and filename != '10.1.1.1':
          path_file='/content/dataset/train/'
        tables = predict('/content/dataset1/'  +filename+'.bmp')
        table = Image.open('/content/dataset1/'  +filename+'.bmp')
        i = 0
        for ymin,xmin,ymax,xmax in tables:
          ajustar(col_mask[ymin-2:ymax+2, xmin-2:xmax+2])
          column = Image.fromarray(col_mask.astype(np.uint8),'L')
          column.crop(box=[xmin-2,ymin-2,xmax+2,ymax+2]).save(path_file+'columns/' + filename + '_' + str(i) + '.png')
          table.crop(box=[xmin-2,ymin-2,xmax+2,ymax+2]).save(path_file+'images/'+filename+ '_' + str(i) +'.png', 'png')
          i+=1
    k+=1

## Comprimiento dataset a un archivo zip para su posterior uso

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
shutil.make_archive('/content/drive/MyDrive/ColumnDataset','zip','/content/dataset')

'/content/Data.zip'